# FF++ Staged Fine-tuning (Step 9)

**Stages:**
- **A**: Head-only stabilization (2 epochs)
- **B**: Partial unfreeze - layer4 (8 epochs)
- **C**: Optional deeper unfreeze (5 epochs)

In [ ]:
# 1. Mount Drive & Clone Repo
from google.colab import drive
drive.mount('/content/drive')

!rm -rf /content/Team-Converge
!git clone https://github.com/Incharajayaram/Team-Converge.git /content/Team-Converge
%cd /content/Team-Converge/Finetune1

In [ ]:
# 2. Install Dependencies
!pip install -q mediapipe pyyaml tqdm gdown scikit-learn

In [ ]:
# 3. Download FF++ Data from Drive
import os

# Your Drive file ID - UPDATE THIS if you re-upload
FILE_ID = "1zdmdO0_rUyMspOkZtAi3b0oN_NOxYyLm"
ZIP_PATH = "/content/ffpp_data.zip"

if os.path.exists(ZIP_PATH):
    os.remove(ZIP_PATH)

!pip install -q --upgrade gdown
!gdown {FILE_ID} --output {ZIP_PATH} --fuzzy

size_gb = os.path.getsize(ZIP_PATH) / 1e9
print(f"Downloaded: {size_gb:.2f} GB")

In [ ]:
# 4. Extract Data
!mkdir -p /content/data/raw/ffpp
!unzip -q /content/ffpp_data.zip -d /content/data/raw/ffpp
print(f"Extracted items: {len(os.listdir('/content/data/raw/ffpp'))}")

In [ ]:
# 5. Verify GPU
import torch
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")
print(f"CUDA available: {torch.cuda.is_available()}")

In [ ]:
# 6. Run Stage A: Head-only stabilization (2 epochs)
!python train_staged.py --config config.yaml \
    --override dataset.ffpp_root=/content/data/raw/ffpp \
    --override caching.cache_dir=/content/cache/faces \
    --stages A \
    --output_dir /content/drive/MyDrive/ffpp_training/staged

In [ ]:
# 7. Run Stage B: Partial unfreeze (layer4)
!python train_staged.py --config config.yaml \
    --override dataset.ffpp_root=/content/data/raw/ffpp \
    --override caching.cache_dir=/content/cache/faces \
    --stages B \
    --resume /content/drive/MyDrive/ffpp_training/staged/best_model.pt \
    --output_dir /content/drive/MyDrive/ffpp_training/staged

In [ ]:
# 8. (Optional) Stage C: Deeper unfreeze - run only if Stage B plateaus
# !python train_staged.py --config config.yaml \
#     --override dataset.ffpp_root=/content/data/raw/ffpp \
#     --override caching.cache_dir=/content/cache/faces \
#     --stages C \
#     --resume /content/drive/MyDrive/ffpp_training/staged/best_model.pt \
#     --output_dir /content/drive/MyDrive/ffpp_training/staged

In [ ]:
# 9. View Training History
import json
import matplotlib.pyplot as plt

history_path = '/content/drive/MyDrive/ffpp_training/staged/training_history.json'
with open(history_path) as f:
    history = json.load(f)

epochs = [h['epoch'] for h in history]
train_loss = [h['train_loss'] for h in history]
val_loss = [h['val_loss'] for h in history]
val_auc = [h.get('val_auc', 0.5) for h in history]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(epochs, train_loss, 'b-', label='Train Loss')
ax1.plot(epochs, val_loss, 'r-', label='Val Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.legend()
ax1.set_title('Loss Curves')

ax2.plot(epochs, val_auc, 'g-', label='Val AUC')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('AUC')
ax2.set_ylim(0.5, 1.0)
ax2.legend()
ax2.set_title('Validation AUC')

plt.tight_layout()
plt.savefig('/content/drive/MyDrive/ffpp_training/training_curves.png', dpi=150)
plt.show()

print(f"Best val_loss: {min(val_loss):.4f}")
print(f"Best val_auc: {max(val_auc):.4f}")

In [ ]:
# 10. Copy final checkpoint to Drive
!cp /content/drive/MyDrive/ffpp_training/staged/best_model.pt \
    /content/drive/MyDrive/ffpp_training/final_model.pt
print("Final model saved to Drive!")